In [3]:
cd ../../

/mnt/sda/otani/Experiments/hidden-challenges-MR


In [4]:
from src import settings
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.toolbox.utils import _nms
from src.toolbox.eval import evaluate, location_error, accumulate_metrics, categorize_results, summarize_results_per_class
from src.toolbox.utils import _nms
from src.toolbox.visualization import plot_performance_per_class, plot_performance_per_duration
from src.toolbox.data_converters import ActivityNetCap2Instances
import seaborn as sns
import json
import pickle as pkl
import neptune
import os
sns.set_style("white")

In [5]:
raw_data = json.load(open("data/raw/activitynet/val_2.json"))
test_data = ActivityNetCap2Instances(raw_data)

In [6]:
project = neptune.init("mayu-ot/moment-retrieval")
if not os.path.exists("tmp/MOM-48/TAN_c3d_test.pkl"):
    exps = project.get_experiments(id="MOM-48")
    print("downloading results ...")
    exps[0].download_artifact("TAN_c3d_test.pkl", "tmp/MOM-48/")
    print("done!")

In [7]:
def postproc_prediction(predictions):
    preds = []
    for p in tqdm(predictions, desc="postproc"):
        query = p[0][0], p[0][1]
        seg = p[1]
        keep = _nms(np.asarray(seg), np.arange(len(seg))[::-1], thresh=0.5, top_k=5)
        seg = [seg[i] for i in keep]
        preds.append((query, seg, np.arange(len(seg))[::-1].tolist()))
    return preds

def eval_preds(pred_file):
    print(f"loading prediction file {pred_file}")
    predictions = pkl.load(open(pred_file, "rb"))
    preds = postproc_prediction(predictions)
    results = evaluate(test_data, preds)
    summary = accumulate_metrics(results)
    return results, summary

results, summary = eval_preds("tmp/MOM-48/TAN_c3d_test.pkl")

loading prediction file tmp/MOM-48/TAN_c3d_test.pkl


evaluating: 100%|██████████| 17031/17031 [00:12<00:00, 1318.54it/s]


R@1 IoU>0.3 0.59
R@1 IoU>0.5 0.44
R@1 IoU>0.7 0.27
R@5 IoU>0.3 0.86
R@5 IoU>0.5 0.77
R@5 IoU>0.7 0.62
R@10 IoU>0.3 0.86
R@10 IoU>0.5 0.77
R@10 IoU>0.7 0.62


In [8]:
# Check relation between success rates and iput video durations
plt.rcParams.update({'font.size': 14})
fig = plot_performance_per_duration(results, test_data, ax=plt.gca())

In [9]:
# per-action performance
from src.toolbox.eval import get_first_action, categorize_results, summarize_results_per_class
from src.toolbox.utils import _load_top_actions
top_actions = _load_top_actions("activitynet")
cat_fn = lambda x: get_first_action(x, top_actions)
categorized_results = categorize_results(results, cat_fn)
metrics_per_cls = summarize_results_per_class(categorized_results)

In [10]:
class_labels = list(metrics_per_cls.keys())
frequent_class = [label for label in class_labels if metrics_per_cls[label]["n_instance"] > 100]
_ = plot_performance_per_class({l: v for l, v in metrics_per_cls.items() if l in frequent_class})